In [1]:
import sys
import os

# Adjust this to point to your project root directory
project_root = os.path.abspath(os.path.join(os.getcwd(), '..\\..'))  # or '..\\..' if you're inside models/
if project_root not in sys.path:
    sys.path.insert(0, project_root)
project_root

'c:\\Users\\weezi\\Repositories\\FIT3164_DS_04'

In [3]:
from itertools import product
from sqlalchemy import extract
from backend.models.elo_formula import calculate_elo
from backend.models.elo_predictor import predict_from_elo, compute_accuracy, compute_log_loss
from backend.db_access.db_base import get_db_session
from backend.db_access.schema import Match
session = get_db_session()
best_accuracy = 0
best_loss = 1
best_params = {}
train_matches = (session.query(Match)
    .filter(extract('year', Match.date) >= 2010)
    .filter(extract('year', Match.date) <= 2022)
    .all())
val_matches = (session.query(Match)
    .filter(extract('year', Match.date) == 2023)
    .all()
    )
for k, init, home in product([10, 20, 30, 40], [950, 1000, 1050], [0, 50, 100]):
    ratings = calculate_elo(
        train_matches,
        k_value=k,
        initial_elo=init,
        home_advantage=home
    )
    # Predict results on validation matches using current ratings
    predictions = predict_from_elo(ratings, val_matches)
    val_dict = {match.id: match for match in val_matches}
    acc = compute_accuracy(predictions, val_dict)
    loss = compute_log_loss(predictions, val_dict)
    
    print(acc, loss, k, init, home)
    if loss < best_loss:
        best_loss = loss
        # best_accuracy = acc
        best_params = {'k': k, 'initial': init, 'home': home}

64.35185185185185 0.6652705417922667 10 950 0
63.888888888888886 0.6599832123793989 10 950 50
63.42592592592593 0.6568427359541793 10 950 100
64.35185185185185 0.6652705417922667 10 1000 0
63.888888888888886 0.6599832123793988 10 1000 50
63.42592592592593 0.6568427359541792 10 1000 100
64.35185185185185 0.6652705417922667 10 1050 0
63.888888888888886 0.6599832123793988 10 1050 50
63.42592592592593 0.6568427359541793 10 1050 100
62.96296296296296 0.6919230935278593 20 950 0
63.42592592592593 0.684796455988998 20 950 50
61.57407407407407 0.6799328323135673 20 950 100
62.96296296296296 0.6919230935278593 20 1000 0
63.42592592592593 0.684796455988998 20 1000 50
61.57407407407407 0.6799328323135673 20 1000 100
62.96296296296296 0.6919230935278594 20 1050 0
63.42592592592593 0.684796455988998 20 1050 50
61.57407407407407 0.6799328323135673 20 1050 100
63.42592592592593 0.7246810080558985 30 950 0
61.57407407407407 0.7173207773149004 30 950 50
60.64814814814815 0.7121698073744135 30 950 100
6

In [5]:
print("Best Params:", best_params)
# print("Best Accuracy:", best_accuracy)
print("Best Loss:", best_loss)


Best Params: {'k': 10, 'initial': 1000, 'home': 100}
Best Loss: 0.6568427359541792
